In [1]:
import sys
import time
import json
import psycopg2
import constants

from riotwatcher import LolWatcher, ValWatcher, ApiError, RiotWatcher

import match_fetcher

In [2]:
API_KEY = constants.API_KEY_SERVICE
db_pass = constants.db_password
db_ip = constants.db_ip
lol_watcher = LolWatcher(API_KEY)
riot_watcher = RiotWatcher(API_KEY)
with open('/Users/Erica/Desktop/Personal Data Projects/Yrden DB/Riot API/queues.json') as file:
    data = json.load(file)

#Connect to Yrden DB
#Private args are executed and then deleted for safety
conn = psycopg2.connect(dbname="YRDEN", user="postgres", password = db_pass, host =db_ip)
cur = conn.cursor()

In [3]:
lol_watcher.data_dragon.versions_for_region('na1')['n']

{'item': '13.24.1',
 'rune': '7.23.1',
 'mastery': '7.23.1',
 'summoner': '13.24.1',
 'champion': '13.24.1',
 'profileicon': '13.24.1',
 'map': '13.24.1',
 'language': '13.24.1',
 'sticker': '13.24.1'}

In [4]:
#
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDitem = DDRegion['item']
DDrune = DDRegion['rune']
DDmastery = DDRegion['mastery']
DDsummoner = DDRegion['summoner']
DDchampion = DDRegion['champion']
DDmap = DDRegion['map']

In [5]:
lol_watcher.data_dragon.summoner_spells(DDsummoner).keys()

dict_keys(['type', 'version', 'data'])

In [6]:
summoners = lol_watcher.data_dragon.summoner_spells(DDsummoner)['data']
print(summoners.keys())
summoners['SummonerSnowURFSnowball_Mark']


dict_keys(['SummonerBarrier', 'SummonerBoost', 'SummonerCherryFlash', 'SummonerCherryHold', 'SummonerDot', 'SummonerExhaust', 'SummonerFlash', 'SummonerHaste', 'SummonerHeal', 'SummonerMana', 'SummonerPoroRecall', 'SummonerPoroThrow', 'SummonerSmite', 'SummonerSnowURFSnowball_Mark', 'SummonerSnowball', 'SummonerTeleport', 'Summoner_UltBookPlaceholder', 'Summoner_UltBookSmitePlaceholder'])


{'id': 'SummonerSnowURFSnowball_Mark',
 'name': 'Mark',
 'description': 'Throw a snowball in a straight line at your enemies. If it hits an enemy, they become marked, granting True Sight, and your champion can quickly travel to the marked target as a follow up.',
 'tooltip': 'Throw a snowball a long distance, dealing {{ tooltipdamagetotal }} true damage to the first enemy unit hit and granting <span class="coloree91d7">True Sight</span> of the target. If it hits an enemy, this ability can be recast for {{ e3 }} seconds to Dash to the tagged unit, dealing an additional {{ tooltipdamagetotal }} true damage. Dashing to the target will reduce the cooldown of Mark by {{ e4 }}%.<br /><br /><span class="colorFFFF00">Mark projectiles are not stopped by spell shields or projectile mitigation.</span>',
 'maxrank': 1,
 'cooldown': [80],
 'cooldownBurn': '80',
 'cost': [0],
 'costBurn': '0',
 'datavalues': {},
 'effect': [None, [10], [5], [3], [25], [0], [0], [0], [0], [0], [0]],
 'effectBurn': [N

In [7]:
with conn.cursor() as curs:
    summoner_dict = {}
    curs.execute(''' DROP TABLE "LOLLOV".lol_summoner_spells ''')
    curs.execute(''' CREATE TABLE "LOLLOV".lol_summoner_spells
    (
    spell_name varchar,
    spell_id int,
    modes varchar
    );''')
    conn.commit()
    for n in summoners:
        summoner_dict.update({'spell_name' : summoners[n]['name'],
                             'spell_id' : summoners[n]['key']})
        curs.execute('''INSERT INTO "LOLLOV".lol_summoner_spells
        (spell_name, spell_id)
        VALUES
        (%(spell_name)s, %(spell_id)s, %(modes)s);''', summoner_dict)
        conn.commit()
    

KeyError: 'modes'

In [8]:
lol_watcher.data_dragon.champions(DDchampion).keys()

dict_keys(['type', 'format', 'version', 'data'])

In [9]:
lol_watcher.data_dragon.champions(DDchampion)['data'].keys()

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Po

In [10]:
time.localtime()

time.struct_time(tm_year=2023, tm_mon=12, tm_mday=16, tm_hour=12, tm_min=40, tm_sec=3, tm_wday=5, tm_yday=350, tm_isdst=0)

In [11]:
lol_watcher.data_dragon.champions(DDchampion)['data'].keys()

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Po

In [12]:
lol_watcher.data_dragon.champions(DDchampion)['data']['Sylas'].keys()

dict_keys(['version', 'id', 'key', 'name', 'title', 'blurb', 'info', 'image', 'tags', 'partype', 'stats'])

In [13]:
lol_watcher.data_dragon.maps(DDmap).keys()

dict_keys(['type', 'version', 'data'])

In [14]:
lol_watcher.data_dragon.maps(DDmap)['data'].keys()

dict_keys(['11', '12', '21', '22', '30'])

In [15]:
lol_watcher.data_dragon.maps(DDmap)['data']['11']

{'MapName': "Summoner's Rift",
 'MapId': '11',
 'image': {'full': 'map11.png',
  'sprite': 'map0.png',
  'group': 'map',
  'x': 0,
  'y': 0,
  'w': 48,
  'h': 48}}

In [16]:
len(lol_watcher.data_dragon.runes_reforged(DDrune))

5

In [17]:
lol_watcher.data_dragon.runes_reforged(DDrune)[0].keys()

dict_keys(['id', 'key', 'name', 'icon', 'slots'])

Iterate through the list 5 times (length of runes_reforged base api call) first. This gives you each tree. Sorc/Dom/Precision/Imag/Resolve
Then go to "Slots" key in dictionary
Iterate through each to get each row of the runes reforged. (four times)
Go to 'Runes' key in dictionary
Iterate through list for each rune (needs to be variable as things will change)
Update dictionary and insert values into table

In [18]:
len(lol_watcher.data_dragon.runes_reforged(DDrune))

5

In [19]:
len(lol_watcher.data_dragon.runes_reforged(DDrune)[1])

5

In [20]:
lol_watcher.data_dragon.runes_reforged(DDrune)[1]

{'id': 8300,
 'key': 'Inspiration',
 'name': 'Inspiration',
 'icon': 'ASSETS/Perks/Styles/7203_Whimsy.dds',
 'slots': [{'runes': [{'id': 8326,
     'key': 'KSAlchemicalEsoterica',
     'name': 'Unsealed Spellbook',
     'shortDesc': 'Exchange Summoner Shards at the shop to change your Summoner Spells during game. Your Summoner Spells have reduced cooldowns. <br>',
     'longDesc': "Gain a Summoner Shard at @ShardFirstMinutes@ min and another every @ShardRechargeMinutes@ min after (Max @Maxshards@ shards).<br><br>While near the shop, you can exchange @ShardCost@ Summoner Shard to replace a Summoner Spell with a different one. <br><br>Additionally, your Summoner Spell Cooldowns are reduced by @SummonerCDR.0*100@%.<br><br><rules><i>Smite:</i> Buying Smite won't grant access to Smite items<br>You cannot have two of the same Summoner Spell</rules>",
     'icon': 'ASSETS/Perks/Styles/Inspiration/KSAlchemicalEsoterica/KSAlchemicalEsoterica.dds'},
    {'id': 8351,
     'key': 'KSGlacialAugment

In [23]:
print(lol_watcher.data_dragon.runes_reforged(DDrune)[1]['slots'][3]['runes'][0]['name'])
lol_watcher.data_dragon.runes_reforged(DDrune)[1]['slots'][0]['runes'][0]['id']

Cosmic Insight


8326

In [22]:
def commit_champ_lov(conn, champions):
    with conn.cursor() as curs:
        for key in champions:
            
            vals = lol_watcher.data_dragon.champions(DDchampion)['data'][key]
            new_vals = {}
            for n in range(len(vals)):
                new_vals.update({'champion_id' : vals['key'],
                                'champion_name' : vals['name']                                 
                                })

                
                curs.execute('''INSERT INTO "LOLLOV".lol_champs 
                        VALUES (%(champion_id)s, %(champion_name)s)
                        ON CONFLICT (champion_id) DO NOTHING
                            ;''', new_vals)
                            
                conn.commit()
        print(curs.rowcount, " LOVs added")

In [24]:
commit_champ_lov(conn, lol_watcher.data_dragon.champions(DDchampion)['data'])

0  LOVs added


In [25]:
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDrune = DDRegion['rune']
runes = lol_watcher.data_dragon.runes_reforged(DDrune)
rows = runes[0]['slots']
for row in range(len(rows)):
    runes = rows[row]['runes']
    print(runes)

[{'id': 8112, 'key': 'StormlordsMandate', 'name': 'Electrocute', 'shortDesc': 'Hitting a champion with 3 <b>separate</b> attacks or abilities in @WindowDuration@s deals bonus <lol-uikit-tooltipped-keyword key="LinkTooltip_Description_AdaptiveDmg">adaptive damage</lol-uikit-tooltipped-keyword>.', 'longDesc': "Hitting a champion with 3 <b>separate</b> attacks or abilities within @WindowDuration@s deals bonus <lol-uikit-tooltipped-keyword key='LinkTooltip_Description_AdaptiveDmg'><font color='#48C4B7'>adaptive damage</font></lol-uikit-tooltipped-keyword>.<br><br>Damage: @DamageBase@ - @DamageMax@ (+@BonusADRatio.-1@ bonus AD, +@APRatio.-1@ AP) damage.<br><br>Cooldown: @Cooldown@ - @CooldownMin@s<br><br><hr></hr><i>'We called them the Thunderlords, for to speak of their lightning was to invite disaster.'</i>", 'icon': 'ASSETS/Perks/Styles/Domination/TLords/TLords.dds'}, {'id': 8124, 'key': 'KSLycanthropy', 'name': 'Predator', 'shortDesc': 'Add an active effect to your boots that grants a l

In [28]:
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDrune = DDRegion['rune']
runes = lol_watcher.data_dragon.runes_reforged(DDrune)
runes[2]['slots']


[{'runes': [{'id': 8005,
    'key': 'PressThreeAttacks',
    'name': 'Press the Attack',
    'shortDesc': 'Hitting an enemy champion @HitsRequired@ consecutive times makes them vulnerable, dealing bonus damage and causing them to take more damage from all sources for @AmpDuration@s.',
    'longDesc': "Hitting an enemy champion with @HitsRequired@ consecutive basic attacks deals @MinDamage@ - @MaxDamage@ bonus <lol-uikit-tooltipped-keyword key='LinkTooltip_Description_AdaptiveDmg'><font color='#48C4B7'>adaptive damage</font></lol-uikit-tooltipped-keyword> (based on level) and makes them vulnerable, increasing the damage they take by @AmpPotencySelf.0*100@% from all sources for @AmpDuration@s.",
    'icon': 'ASSETS/Perks/Styles/Precision/PressThreeAttacks/PressThreeAttacks.dds'},
   {'id': 8008,
    'key': 'Berserk',
    'name': 'Lethal Tempo',
    'shortDesc': '@LeadInDelay.1@s after damaging a champion gain a large amount of Attack Speed. Lethal Tempo allows you to temporarily exceed t

In [29]:
for i, l in enumerate(rune_trees):
    print(rune_trees['slots'])

NameError: name 'rune_trees' is not defined

In [30]:
for n, i in enumerate(runes):
    rune_trees = runes[n]
    for l, k in enumerate(rune_trees['key']):
        print(l)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
0
1
2
3
4
5
6


In [ ]:
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDrune = DDRegion['rune']
runes = lol_watcher.data_dragon.runes_reforged(DDrune)

runes[0]['key']



In [31]:
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDrune = DDRegion['rune']
runes = lol_watcher.data_dragon.runes_reforged(DDrune)
for n in range(len(runes)):
    print(n)
    mastery_tree = runes[n]['key']
    print(mastery_tree)

0
Domination
1
Inspiration
2
Precision
3
Resolve
4
Sorcery


In [32]:
DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
DDrune = DDRegion['rune']
runes = lol_watcher.data_dragon.runes_reforged(DDrune)

runes[0]['slots'][0]['runes'][1]['name']

for n in range(len(runes)):
    print(runes[n]['slots'])

[{'runes': [{'id': 8112, 'key': 'StormlordsMandate', 'name': 'Electrocute', 'shortDesc': 'Hitting a champion with 3 <b>separate</b> attacks or abilities in @WindowDuration@s deals bonus <lol-uikit-tooltipped-keyword key="LinkTooltip_Description_AdaptiveDmg">adaptive damage</lol-uikit-tooltipped-keyword>.', 'longDesc': "Hitting a champion with 3 <b>separate</b> attacks or abilities within @WindowDuration@s deals bonus <lol-uikit-tooltipped-keyword key='LinkTooltip_Description_AdaptiveDmg'><font color='#48C4B7'>adaptive damage</font></lol-uikit-tooltipped-keyword>.<br><br>Damage: @DamageBase@ - @DamageMax@ (+@BonusADRatio.-1@ bonus AD, +@APRatio.-1@ AP) damage.<br><br>Cooldown: @Cooldown@ - @CooldownMin@s<br><br><hr></hr><i>'We called them the Thunderlords, for to speak of their lightning was to invite disaster.'</i>", 'icon': 'ASSETS/Perks/Styles/Domination/TLords/TLords.dds'}, {'id': 8124, 'key': 'KSLycanthropy', 'name': 'Predator', 'shortDesc': 'Add an active effect to your boots that

In [ ]:
def commit_rune_lov(conn):
    with conn.cursor() as curs:
        DDRegion = lol_watcher.data_dragon.versions_for_region('na1')['n']
        DDrune = DDRegion['rune']
        runes = lol_watcher.data_dragon.runes_reforged(DDrune)
        for n in range(len(runes)):
            
            vals = runes[n]
            tree = vals['key']
            new_vals = {}
            for n in range(len(vals)):
                new_vals.update({'rune_id' : vals['key'],
                                'rune_name' : vals['name'],
                                'rune_tree' : vals['tree']
                                })

                
                curs.execute('''INSERT INTO "LOLLOV".lol_runes 
                        VALUES (%(rune_id)s, %(rune_name)s)
                        ON CONFLICT (rune_id) DO NOTHING
                            ;''', new_vals)
                            
                conn.commit()
        print(curs.rowcount, " LOVs added")

In [ ]:
queue_dict = {}
with conn.cursor() as curs:
    for n in data:
        curs.execute('''INSERT INTO "LOLLOV".lol_queues
                (queue_id, map, description)
                VALUES
                (%(queueId)s, %(map)s, %(description)s);''', n)
        conn.commit()